In [1]:
import os
os.chdir(r'C:\Users\빅데이터활용센터\Desktop\신천')
from shapely.geometry import Point, LineString, Polygon
import geopandas as gpd
import pandas as pd
from fiona.crs import from_string
from shapely.geometry import MultiPoint
from pyproj import Proj, transform

In [2]:
#유동인구 데이터 가져오기

final_sum = pd.read_csv('final_sum.csv')
final_mean = pd.read_csv('final_mean.csv')

epsg5179 = Proj(init="epsg:5179")
epsg4329 = Proj(init="epsg:4329")

#5179 좌표계 -> 5181 좌표계             
final_sum['x'], final_sum['y'] = transform(epsg5179, epsg4329, final_sum['X_COORD'].tolist(), final_sum['Y_COORD'].tolist())
final_mean['x'], final_mean['y'] = transform(epsg5179, epsg4329, final_mean['X_COORD'].tolist(), final_mean['Y_COORD'].tolist())

C:\ProgramData\Anaconda3\lib\site-packages\pyproj\crs\crs.py:131: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  in_crs_string = _prepare_from_proj_string(in_crs_string)
C:\ProgramData\Anaconda3\lib\site-packages\pyproj\crs\crs.py:131: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  in_crs_string = _prepare_from_proj_string(in_crs_string)
<ipython-input-2-1aa170bc4b18>:10: DeprecationWarning: This function is deprecated. See: https://pyproj4.github.io/pyproj/stable/gotchas.html#upgrading-to-pyproj-2-from-pyproj-1
  final_sum['x'], final_sum['y'] = transfo

In [3]:
#gpd로 만들기
sumPoint = gpd.GeoDataFrame(final_sum, 
                           geometry = gpd.points_from_xy(final_sum.x, final_sum.y),
                          crs='epsg:4329')

meanPoint = gpd.GeoDataFrame(final_mean, 
                           geometry = gpd.points_from_xy(final_mean.x, final_mean.y),
                          crs='epsg:4329')

In [7]:
L_bridge = ['가창', '경대칠성', '대봉희망', '도청경대', '동신수성', '두산용두', '상동두산', '성북도청', 
            '수성대봉', '신천동신', '용두파동', '중동상동', '칠성신천', '침산', '침산성북', '파동가창', 
            '희망중동']

final_df = pd.DataFrame()

for bridge in L_bridge :
    
    #데이터 읽어오기
    url = './교량별 15분/' + bridge + ' final15.csv'
    df = pd.read_csv(url)
    df = gpd.GeoDataFrame(df)

    df['lon'] = df['lon'].astype(float)
    df['lat'] = df['lat'].astype(float)
    
    #gdf로 변환
    df['geometry'] = df.apply(lambda row : Point([row['lon'], row['lat']]), axis=1)
    gdf = gpd.GeoDataFrame(df[['osmid', 'street_count', 'geometry']], geometry='geometry', crs = 'epsg:4329')

    #추출된 df의 점들로 폴리곤 생성, 경계값만 가져와서 저장
    poly = Polygon(Polygon(MultiPoint([[p.x, p.y] for p in df.geometry]).convex_hull).exterior)
    
    #교각명
    name = bridge
    
    #전체 통합
    final_df = final_df.append(pd.DataFrame([name, poly]).T, ignore_index=True)
    

final_df.columns=['name', 'geometry']
final_gdf = gpd.GeoDataFrame(final_df, geometry='geometry', crs='epsg:4329')
final_gdf

,name,geometry
0,가창,"POLYGON ((128.58400 35.89977, 128.58291 35.900..."
1,경대칠성,"POLYGON ((128.60581 35.86899, 128.60210 35.869..."
2,대봉희망,"POLYGON ((128.61472 35.84350, 128.61354 35.843..."
3,도청경대,"POLYGON ((128.59590 35.88019, 128.59317 35.883..."
4,동신수성,"POLYGON ((128.61729 35.85601, 128.60014 35.859..."
5,두산용두,"POLYGON ((128.61680 35.82864, 128.61427 35.830..."
6,상동두산,"POLYGON ((128.60293 35.82590, 128.59636 35.832..."
7,성북도청,"POLYGON ((128.59493 35.88242, 128.58999 35.882..."
8,수성대봉,"POLYGON ((128.61692 35.85212, 128.59946 35.852..."
9,신천동신,"POLYGON ((128.62406 35.86273, 128.60496 35.867..."


In [12]:
final_gdf.to_csv('bridge_polygon.csv')

In [9]:
#spatail join
sumTime = gpd.sjoin(sumPoint, final_gdf)

#relocate columns
sumTime = sumTime[['name', 'month', 'geometry',
            'H_M_10', 'H_M_20', 'H_M_30', 'H_M_40','H_M_50', 'H_M_60', 'H_M_70', 
            'H_W_10', 'H_W_20', 'H_W_30', 'H_W_40', 'H_W_50', 'H_W_60', 'H_W_70', 
            'W_M_10', 'W_M_20', 'W_M_30', 'W_M_40', 'W_M_50', 'W_M_60', 'W_M_70', 
            'W_W_10', 'W_W_20', 'W_W_30', 'W_W_40', 'W_W_50', 'W_W_60', 'W_W_70', 
            'V_M_10', 'V_M_20', 'V_M_30', 'V_M_40', 'V_M_50', 'V_M_60', 'V_M_70',
            'V_W_10', 'V_W_20', 'V_W_30', 'V_W_40','V_W_50', 'V_W_60', 'V_W_70']].reset_index(drop=True)
       
sumTime

,name,month,geometry,H_M_10,H_M_20,H_M_30,H_M_40,H_M_50,H_M_60,H_M_70,...,V_M_50,V_M_60,V_M_70,V_W_10,V_W_20,V_W_30,V_W_40,V_W_50,V_W_60,V_W_70
0,성북도청,1,POINT (128.58231 35.88960),6.05,7.84,6.78,7.40,9.18,9.13,6.26,...,4.67,3.22,1.25,1.08,1.36,1.52,2.19,2.81,2.17,1.30
1,성북도청,1,POINT (128.58232 35.89005),2.20,2.15,1.89,2.12,2.87,2.74,1.41,...,1.50,0.97,0.31,0.60,0.41,0.50,0.69,0.89,0.50,0.31
2,성북도청,1,POINT (128.58286 35.88914),14.11,18.41,15.89,17.45,21.40,21.43,14.62,...,10.93,7.63,2.91,2.55,3.19,3.61,5.09,6.62,5.13,3.08
3,성북도청,1,POINT (128.58287 35.88959),59.83,77.96,67.32,74.04,90.77,90.85,62.03,...,46.36,32.18,12.40,10.70,13.62,15.40,21.53,27.95,21.70,12.98
4,성북도청,1,POINT (128.58287 35.89004),6.73,6.38,5.75,6.40,8.84,8.34,4.30,...,4.64,2.95,1.00,1.80,1.34,1.57,2.17,2.85,1.47,0.89
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
162003,두산용두,12,POINT (128.61748 35.82932),26.49,18.77,18.46,21.64,21.23,15.75,11.18,...,12.64,6.75,3.05,8.22,11.70,12.05,13.68,12.78,7.68,3.96
162004,두산용두,12,POINT (128.61748 35.82977),70.16,49.76,48.91,57.37,56.34,41.79,29.63,...,33.54,17.92,8.04,21.81,31.05,31.86,36.25,33.86,20.42,10.58
162005,두산용두,12,POINT (128.61749 35.83022),26.57,18.83,18.49,21.71,21.32,15.84,11.21,...,12.73,6.78,3.06,8.27,11.76,12.10,13.75,12.80,7.71,3.98
162006,두산용두,12,POINT (128.61803 35.82932),11.03,7.77,7.65,9.00,8.86,6.56,4.69,...,5.26,2.80,1.29,3.41,4.87,4.99,5.70,5.34,3.21,1.65


In [10]:
#spatail join
meanTime = gpd.sjoin(meanPoint, final_gdf)

#relocate columns
meanTime = meanTime[['name', 'month', 'geometry',
            'H_M_10', 'H_M_20', 'H_M_30', 'H_M_40','H_M_50', 'H_M_60', 'H_M_70', 
            'H_W_10', 'H_W_20', 'H_W_30', 'H_W_40', 'H_W_50', 'H_W_60', 'H_W_70', 
            'W_M_10', 'W_M_20', 'W_M_30', 'W_M_40', 'W_M_50', 'W_M_60', 'W_M_70', 
            'W_W_10', 'W_W_20', 'W_W_30', 'W_W_40', 'W_W_50', 'W_W_60', 'W_W_70', 
            'V_M_10', 'V_M_20', 'V_M_30', 'V_M_40', 'V_M_50', 'V_M_60', 'V_M_70',
            'V_W_10', 'V_W_20', 'V_W_30', 'V_W_40','V_W_50', 'V_W_60', 'V_W_70']].reset_index(drop=True)

meanTime

,name,month,geometry,H_M_10,H_M_20,H_M_30,H_M_40,H_M_50,H_M_60,H_M_70,...,V_M_50,V_M_60,V_M_70,V_W_10,V_W_20,V_W_30,V_W_40,V_W_50,V_W_60,V_W_70
0,성북도청,1,POINT (128.58231 35.88960),0.195161,0.252903,0.218710,0.238710,0.296129,0.294516,0.201935,...,0.150645,0.103871,0.040323,0.034839,0.043871,0.049032,0.070645,0.090645,0.070000,0.041935
1,성북도청,1,POINT (128.58232 35.89005),0.070968,0.069355,0.060968,0.068387,0.092581,0.088387,0.045484,...,0.048387,0.031290,0.010000,0.019355,0.013226,0.016129,0.022258,0.028710,0.016129,0.010000
2,성북도청,1,POINT (128.58286 35.88914),0.455161,0.593871,0.512581,0.562903,0.690323,0.691290,0.471613,...,0.352581,0.246129,0.093871,0.082258,0.102903,0.116452,0.164194,0.213548,0.165484,0.099355
3,성북도청,1,POINT (128.58287 35.88959),1.930000,2.514839,2.171613,2.388387,2.928065,2.930645,2.000968,...,1.495484,1.038065,0.400000,0.345161,0.439355,0.496774,0.694516,0.901613,0.700000,0.418710
4,성북도청,1,POINT (128.58287 35.89004),0.217097,0.205806,0.185484,0.206452,0.285161,0.269032,0.138710,...,0.149677,0.095161,0.032258,0.058065,0.043226,0.050645,0.070000,0.091935,0.047419,0.028710
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
162003,두산용두,12,POINT (128.61748 35.82932),0.854516,0.605484,0.595484,0.698065,0.684839,0.508065,0.360645,...,0.407742,0.217742,0.098387,0.265161,0.377419,0.388710,0.441290,0.412258,0.247742,0.127742
162004,두산용두,12,POINT (128.61748 35.82977),2.263226,1.605161,1.577742,1.850645,1.817419,1.348065,0.955806,...,1.081935,0.578065,0.259355,0.703548,1.001613,1.027742,1.169355,1.092258,0.658710,0.341290
162005,두산용두,12,POINT (128.61749 35.83022),0.857097,0.607419,0.596452,0.700323,0.687742,0.510968,0.361613,...,0.410645,0.218710,0.098710,0.266774,0.379355,0.390323,0.443548,0.412903,0.248710,0.128387
162006,두산용두,12,POINT (128.61803 35.82932),0.355806,0.250645,0.246774,0.290323,0.285806,0.211613,0.151290,...,0.169677,0.090323,0.041613,0.110000,0.157097,0.160968,0.183871,0.172258,0.103548,0.053226


In [13]:
sumFinal = sumTime.groupby(['name', 'month']).sum().reset_index(drop=False)
meanFinal = meanTime.groupby(['name', 'month']).mean().reset_index(drop=False)

In [17]:
sumFinal.to_csv('bridge_sum.csv', encoding='cp949', index=False)
meanFinal.to_csv('bridge_mean.csv', encoding='cp949', index=False)

In [18]:
myFinal = meanTime.groupby(['name', 'month']).sum().reset_index(drop=False)
myFinal

,name,month,H_M_10,H_M_20,H_M_30,H_M_40,H_M_50,H_M_60,H_M_70,H_W_10,...,V_M_50,V_M_60,V_M_70,V_W_10,V_W_20,V_W_30,V_W_40,V_W_50,V_W_60,V_W_70
0,가창,1,287.050968,256.137419,191.900000,227.680645,288.670000,251.436774,157.846129,232.382258,...,174.258387,106.040000,40.690323,88.679677,71.547097,85.800000,117.109032,125.811613,76.342581,41.913548
1,가창,2,298.955000,266.456786,186.068929,240.294286,301.141786,251.417857,157.691071,245.119286,...,181.392143,109.189286,45.208214,94.677500,76.032857,95.599643,128.117143,129.606071,79.837857,44.582857
2,가창,3,255.485806,254.666129,180.523226,225.723871,281.240968,221.297419,137.022903,213.589032,...,181.076129,108.615806,45.495161,84.648710,73.970323,92.253871,125.520323,128.520323,78.935484,45.098065
3,가창,4,264.335333,258.282333,184.260667,234.656000,290.850000,226.302333,137.591000,217.028000,...,198.316333,119.204000,48.561667,86.568000,78.155667,96.432667,132.756333,139.303333,83.830333,47.618667
4,가창,5,268.128387,264.820000,185.201935,234.157419,302.978387,238.537097,141.045484,222.587097,...,194.877419,118.227742,48.250645,95.661935,85.606129,104.921613,141.820968,143.455806,88.378065,47.458065
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
187,희망중동,8,1667.248065,1192.590968,1314.458065,1522.805484,1515.448065,1284.177097,1278.096774,1494.212258,...,546.034516,408.704194,282.520968,457.902903,383.040323,476.289032,560.948387,561.481613,464.703548,351.382258
188,희망중동,9,1553.537333,1209.572333,1292.486000,1522.363667,1509.683333,1252.381333,1253.037667,1410.600000,...,571.341667,403.916333,279.924333,433.997000,391.882000,512.731333,568.496333,555.067000,443.439333,315.946333
189,희망중동,10,1589.108065,1225.031613,1333.609677,1528.946452,1514.148065,1228.141613,1196.525806,1483.295161,...,579.728065,416.566129,289.924194,425.583226,400.320000,509.314194,588.116129,574.239032,464.049032,344.436774
190,희망중동,11,1556.311667,1183.511333,1299.127667,1485.183333,1487.984667,1184.703000,1164.546667,1458.093667,...,660.807000,463.769000,333.693000,512.472333,463.601667,606.078667,754.125000,680.915667,544.368000,423.079333
